# LAB 2 Andrea Mirenda s327644

The way to install any library using a linux terminal aproach is the following:

In [25]:
import os
os.system("pip install pandas")

1

Import with predefined python environment

In [26]:
import pandas as pd
from collections import Counter
import random
import math
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from itertools import combinations
from tqdm.auto import tqdm
import geopy.distance
from geopy.distance import geodesic
import networkx as nx
from collections import deque

# data import
We use pandas to import and use the data about the cities.
Each city has the coordinates (x and y) which allows us to compute the Euclidean distance between 2 cities.

In [27]:
cities = pd.read_csv('cities/italy.csv', header=None, names=['City', 'x', 'y'])
cities

,City,x,y
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34
5,Bolzano,46.50,11.35
6,Brescia,45.55,10.22
7,Cagliari,39.22,9.10
8,Catania,37.50,15.08
9,Ferrara,44.84,11.61


# Helping function

Define a function that extract the coordinates of a given city

In [28]:
def find_coordinates(name_city):
    x_coordinate = cities.loc[cities['City'] == name_city, 'x'].values[0]
    y_coordinate = cities.loc[cities['City'] == name_city, 'y'].values[0]
    return (x_coordinate, y_coordinate)

Define a function that computes the Euclidian distance between two cities

In [29]:
def euclidean_distance_math(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    distanza = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return round(distanza, 3)

Define the distance using the geopy function for more accurate distance computation:

In [30]:
def euclidean_distance(c1, c2):
    x1, y1 = c1
    x2, y2 = c2
    return geopy.distance.geodesic((x1, y1), (x2, y2)).km

Define a slightly different one that take as input the name of the cities:

In [31]:
#We define the dictionary that contains the link between the city and the coordinates:
city_coordinates = cities.set_index('City')[['x', 'y']].apply(tuple, axis=1).to_dict()

def euclidean_distance_name(name_c1, name_c2):
    c1 = city_coordinates.get(name_c1)
    c2 = city_coordinates.get(name_c2)
    x1, y1 = c1
    x2, y2 = c2
    return geopy.distance.geodesic((x1, y1), (x2, y2)).km

Function that return the closest city

In [32]:
def closest_city(coord, cities_df):
    closest_city = None
    min_distance = float('inf')
    for _, row in cities_df.iterrows():
        city_coord = (row['x'], row['y'])
        distance = euclidean_distance(coord, city_coord)
        
        if distance < min_distance:
            min_distance = distance
            closest_city = row['City']
    
    return closest_city, round(min_distance, 2)

Function that return the n closest cities to a given one. This may be helpful in the case the city we want to explore has already been explored.

In [33]:
def closest_cities(coord, cities_df, n):
    distanze_citta = []
    for _, row in cities_df.iterrows():
        city_coord = (row['x'], row['y'])
        distanza = euclidean_distance(coord, city_coord)
        distanze_citta.append((row['City'], round(distanza, 2)))
    
    distanze_citta_ordinate = sorted(distanze_citta, key=lambda x: x[1])
    
    return distanze_citta_ordinate[:n]

# Simple greedy aproach

In this simplified version of the problem we can take account of the cities we have already visited and, until the number of elements inside this set is different from the number of cities of the country considered we simply add the closest city to the last city we visited.

In [34]:
# select random city as a starting point
starting_city = cities.sample(n=1)['City'].values[0]

# mantain a set of cities already visited:
visited = set()
distance = 0 #euclidean distance computed
num_visited = 0
num_cities = cities.shape[0]
current_city = find_coordinates(starting_city) # current city esplored
print("numero di città", num_cities)
found = True
num_next_cities = int(num_cities/5)

# while i did not visited all the cities i keep on visiting:
while num_visited<num_cities:
    # if we didn't find the closest one and we did not see all the cities yet
    # we increase the number of close cities to consider.
    if not found:
        num_next_cities = num_cities#increase the number of close cities to consider
    next_city = None
    found = False 
    #take the list of the first num_next_cities cities:
    
    closest_ones = closest_cities(current_city, cities, num_next_cities)
    for city in closest_ones:
        city = city[0]
        if city not in visited:
            found = True
            coordinate_of_next = find_coordinates(city)
            distance += euclidean_distance(current_city, coordinate_of_next)
            current_city = find_coordinates(city)
            num_visited += 1
            visited.add(city)
            break
    #at the end we must end up to the first position:
    if num_visited==num_cities:
        #compute the distance to go from current city to the first one:
        distance += euclidean_distance(current_city, find_coordinates(starting_city))

#we visited all the cities with cost distance:
print("After visiting all the", len(cities), "cities the cost was: ", round(distance, 3), "Km")    
    

numero di città 46
After visiting all the 46 cities the cost was:  4740.094 Km


# Version 2 greedy aproach
As a first improvement we may decide to avoid computing each time find_coordinates, by saving into a dict the link between a city and his cordinates.

We may also decide avoiding to consider at each iterations all the cities, but considering anly the ones we didn't selected yet.
However these changes will improve only the computation, but not the final cost as number of kilometers.

We first define the matrix of distances between cities:

In [35]:
DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.x, c1.y), (c2.x, c2.y)
    ).km

We can now change the "closest_cicties" method:

In [36]:
#create a mapping between index and name of city:
city_to_index = {city: idx for idx, city in enumerate(cities['City'])}

def closest_cities(city_name, cities_df, n):
    city_idx = city_to_index.get(city_name)
    # distances to the city:
    distances = DIST_MATRIX[city_idx]
    # order escluding the same city
    nearest_indices = np.argsort(distances)[1:n+1]
    # get the name of the cities:
    nearest_cities = [(cities.iloc[i]['City'], distances[i]) for i in nearest_indices]
    return nearest_cities

We can also change the find coordinates method (in this way we should optimize a bit the process):

In [37]:
def find_coordinates(name_city):
    return city_coordinates.get(name_city)

The new version of greedy aproach is the following:

In [38]:
# select random city as a starting point
starting_city = cities.sample(n=1)['City'].values[0]

# mantain a set of cities already visited:
visited = set()
distance = 0 #euclidean distance computed
num_visited = 0
num_cities = cities.shape[0]
current_city = find_coordinates(starting_city) # current city esplored
name_current=starting_city
print("numero di città", num_cities)
found = True
num_next_cities = int(num_cities/5)

# while i did not visited all the cities i keep on visiting:
while num_visited<num_cities:
    # if we didn't find the closest one and we did not see all the cities yet
    # we increase the number of close cities to consider.
    if not found:
        num_next_cities = num_cities#increase the number of close cities to consider
    next_city = None
    found = False 
    #take the list of the first num_next_cities cities:
    
    closest_ones = closest_cities(name_current, cities, num_next_cities)
    for city in closest_ones:
        city = city[0]
        if city not in visited:
            found = True
            coordinate_of_next = find_coordinates(city)
            distance += euclidean_distance(current_city, coordinate_of_next)
            current_city = find_coordinates(city)
            name_current = city
            num_visited += 1
            visited.add(city)
            break
    #at the end we must end up to the first position:
    if num_visited==num_cities:
        #compute the distance to go from current city to the first one:
        distance += euclidean_distance(current_city, find_coordinates(starting_city))

#we visited all the cities with cost distance:
print("After visiting all the", len(cities), "cities the cost was: ", round(distance, 3), "Km")    
    

numero di città 46
After visiting all the 46 cities the cost was:  5131.727 Km


# Version 3 greedy aproach


In this version we would like to change the representation of the problem in a way that a solution is represented by a list of tuple object that contains the edge and the cost.

The idea is to:

0. Define all the possible segments as [({c1, c2}, dist)]
1. Start from a city
2. Select minimum segment
3. Add the other extreme of the segment to visited
4. Update the distance
5. Turn back to point 2

Continue this process till len(visited)=num_cities

In [39]:
def fitness(solution):
    tot_dist=0
    for edge in solution:
        tot_dist -= DIST_MATRIX[edge[0], edge[1]]
    
    return (len(solution), tot_dist)

In [40]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]
print(segments)
edges = set()

[({0, 1}, 349.30401144305745), ({0, 2}, 391.0418681947875), ({0, 3}, 383.0204391314121), ({0, 4}, 199.89962990855176), ({0, 5}, 364.0447808729763), ({0, 6}, 338.8096401206489), ({0, 7}, 609.696280667211), ({0, 8}, 690.431902252931), ({0, 9}, 204.4253666122213), ({0, 10}, 183.29551097355267), ({0, 11}, 290.4028573916701), ({0, 12}, 136.70372473443274), ({0, 13}, 377.57973448944483), ({0, 14}, 301.99049212290043), ({0, 15}, 241.85490798374872), ({0, 16}, 258.5284939080245), ({0, 17}, 625.0755286998198), ({0, 18}, 401.9399754108604), ({0, 19}, 237.16017652663953), ({0, 20}, 401.4539129529872), ({0, 21}, 312.00387878900796), ({0, 22}, 438.9328915074273), ({0, 23}, 239.268808375456), ({0, 24}, 608.6754823442737), ({0, 25}, 287.5153079129395), ({0, 26}, 105.17159317964676), ({0, 27}, 139.23847120123253), ({0, 28}, 345.1318096169963), ({0, 29}, 196.77126979341847), ({0, 30}, 137.84822902835992), ({0, 31}, 635.9878049331178), ({0, 32}, 260.6010504730683), ({0, 33}, 90.5967461797061), ({0, 34},

In [41]:
def find_closest(segments, city, visited):
    # Filtra i segmenti che contengono 'city' e hanno un'altra città non in 'visited'
    candidates = [
        (pair, distance) for pair, distance in segments
        if city in pair and (other_city := (pair - {city}).pop()) not in visited
    ]
    closest_segment = min(candidates, key=lambda x: x[1])
    closest_city = (closest_segment[0] - {city}).pop()
    return closest_city, closest_segment[1]

Perform the actual algoithm:

In [42]:
def find_closest(segments, city, visited):
    # Filtra i segmenti che contengono 'city' e hanno un'altra città non in 'visited'
    candidates = [
        (pair, distance) for pair, distance in segments
        if city in pair and (other_city := (pair - {city}).pop()) not in visited
    ]
    closest_segment = min(candidates, key=lambda x: x[1])
    closest_city = (closest_segment[0] - {city}).pop()
    return (int(city), closest_city)

In [43]:
#select first solution:
edges = []
city = 0
visited = []
visited.append(int(city))
while len(visited)<len(cities):
    c0, c1 = find_closest(segments, city, visited)
    edges.append((c0, c1))
    visited.append(c1)
    city=c1
edges.append((edges[len(edges)-1][1], edges[0][0]))
print("distanza finale prima di HC è ", fitness(edges)[1]*-1)

distanza finale prima di HC è  4436.03176952516


This slightly increases the performance of the model.

# Hill climbing

We can now consider using the hill climbing approach. 
As a fitness function we may decide to use a tuple as: (number of covered cities, total distance).

But, we should also consider the fact that a certain solution may be "unfeasable" for our problem because creates a cycle. In this case we may consider returning as first element of the tuple 0.

In [44]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

We consider a solution to be a set of edges that have been selected for the shortest path. 
Something like [({1,2}, cost)], so a list of 
The fitness function 

We apply the final greedy aproach to find the starting solution. From there we would like to tweak the solution to find eventually better solutions. 

We consider a solution to be a list of edges, indicated as tuple of nodes, something like [(2,3), (3,4),(4,19)] ecc.

Now we can try to apply simulated annealing, and the tweak function to find a better solution.

Intuition of the tweak:
    [(TO, MI), (MI, RM), (RM, VV)]
    intermediate:
    [(MI, RM), (RM, VV), (TO, MI)]
    next step:
    [(MI, RM), (RM, VV), (VV, TO)]
    MOST general case:
    [(TO, MI), (MI, RM), (RM, VV), (VV, ME), (ME, SI)]
    index = 1; index2 = 3
    [(TO, MI), (VV, ME), (RM, VV), (MI, RM), (ME, SI)]
    now we change the solution:
    [(TO, MI), (MI, ME), (ME, VV), (VV, RM), (RM, SI)].
    In practice we are first swapping the 2 random elements
    then we have to continuosly update the first extreme of the segment
    and put the second extreme of the previous step. Not change the second one.
    this should happen till the end of the list solution
    Now we change the first element of each tuple staring from index to the end:

In [45]:
def swap_mutation(solution):
    solution = solution.copy()
    # choose the index:
    index = random.randint(1, len(solution) - 2) #not the first or the last one
    index2=index #should be higher
    while index2<=index:
        index2 = random.randint(0, len(solution) - 1)

    # now we have the 2 indeces to use.

    selected_edge1 = solution[index]
    selected_edge2 = solution[index2]

    solution[index] = selected_edge2
    solution[index2] = selected_edge1

    for i in range(index, len(solution)):
        i_elem=solution[i]
        prev_elem=solution[i-1]
        elem = prev_elem[1]
        solution[i]=(elem, i_elem[1])

    return solution

In [46]:
def hill_climbing_tsp_scramble(initial_solution, max_iters=1000):
    best_solution = initial_solution
    fitness_best= fitness(best_solution)

    for _ in range(max_iters):
        new_solution = swap_mutation(best_solution)
        fitness_new= fitness(new_solution)
        if fitness_new > fitness_best:
            best_solution = new_solution
            fitness_best = fitness_new

    return best_solution

# Solution with Simulated annealing

If the number of improvements is increasing we increase the temperatura, otherwise we decrease it.

In [47]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]

#select first solution:
edges = []
city = 32
visited = []
visited.append(int(city))
while len(visited)<len(cities):
    c0, c1 = find_closest(segments, city, visited)
    edges.append((c0, c1))
    visited.append(c1)
    city=c1
edges.append((edges[len(edges)-1][1], edges[0][0]))
print("distanza finale prima di HC è ", fitness(edges)[1]*-1)

distanza finale prima di HC è  4699.058487654932


In [48]:
#simulated annealing
temperatura_iniziale = 100
tasso_riscaldamento = 1.1 
it=0

#one out of five approach
miglioramenti_recenti = deque(maxlen=5) 
miglioramenti_richiesti = 1  

#stopping criteria:
miglioramenti_recenti_stop = deque(maxlen=100)
miglioramenti_recenti_stop.append(True)

# Initial solution: greedy one!
x_corrente = edges
costo_corrente = fitness(edges)

temperatura = temperatura_iniziale
while it<1000:
    it+=1
    # tweak the solution
    x_nuovo = hill_climbing_tsp_scramble(x_corrente)
    costo_nuovo = fitness(x_nuovo)
    
    # variation of fitness by changing sign
    delta_costo = costo_nuovo[1]*(-1) - costo_corrente[1]*(-1) 
    #we are sure the solution after swap mutation is valid if the previous was it.
        
    if delta_costo < 0 or (random.random() < math.exp(-delta_costo / temperatura) and delta_costo!=0):
        x_corrente = x_nuovo
        costo_corrente = costo_nuovo
        miglioramenti_recenti.append(True)
        miglioramenti_recenti_stop.append(True)
    else:
        miglioramenti_recenti.append(False)
        miglioramenti_recenti_stop.append(False)

    if miglioramenti_recenti.count(True) > miglioramenti_richiesti:
        temperatura *= tasso_riscaldamento  # more exploration
    if miglioramenti_recenti.count(True) < miglioramenti_richiesti:
        temperatura /= tasso_riscaldamento
    
    if miglioramenti_recenti_stop.count(True)==0: #stop condition
        break

fitness_new=fitness(x_corrente)[1]*(-1)
print("solution after SA is: ", fitness_new)


solution after SA is:  4554.782443247068


Something else must be still done!

# Improved fitness landscape


We divide the process into 2 steps: 

1. we continue adding the shortest segments and the corrisponding cities to visited. we continue doing it till visited become a vector of all true elements.
2. For each city that is present in the state only once (either on the left or on the right) we take the shortest segment to another city that has the same property.

In [49]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]

#first step:
#select first solution:
edges = [] #current solution
visited = [False] * len(cities) #list of boolean
while not np.all(visited):
    #take the shortest edge:
    min_index = min(range(len(segments)), key=lambda i: segments[i][1])
    min_segment = segments[min_index]
    segments.pop(min_index) #we delete so that we do not select later!
    c1, c2 = list(min_segment[0])
    if not(any(c1 in coppia for coppia in edges) and any(c2 in coppia for coppia in edges)):
        edges.append((c1, c2))
        visited[c1] = True 
        visited[c2] = True

#second step:
#we take the nodes that are present only once:
counter = Counter(element for coppia in edges for element in coppia)
unique_cities = [element for element, count in counter.items() if count == 1]

filtered_segments = [
    segmento for segmento in segments
    if all(city in unique_cities for city in segmento[0])
]
sortered_filtered_segments = sorted(filtered_segments, key=lambda x: x[1])

while len(unique_cities)!=0:
    segment =  sortered_filtered_segments.pop(0)
    c1, c2 = list(segment[0])
    if c1 in unique_cities and c2 in unique_cities:
        edges.append((c1, c2))
        visited[c1] = True 
        visited[c2] = True
        unique_cities.remove(c1)
        unique_cities.remove(c2)


#edges.append((edges[len(edges)-1][1], edges[0][0]))
print("final distance is ", fitness(edges)[1]*-1)
# print(edges)

final distance is  5569.304435077125


Even if it seemed to be a very good idea, it is not.

Another idea could be to consider a solution not as a list of tuple object, but a list of cities:

from: [(0,1), (1,2),(2,3)]
to: [0,1,2,3]

which significately simplify the structure!

In [50]:
def fitness(solution):
    tot_dist=0
    for node in range(len(solution)-1):
        tot_dist -= DIST_MATRIX[solution[node], solution[node+1]]
    
    return (len(solution), tot_dist)

The advantage of this representation is the incredible ability to create simple tweak functions.

In [51]:
def swap_mutation(solution):
    # choose the index:
    #print("soluzione iniziale:", fitness(solution))
    index = random.randint(1, len(solution) - 2) #not the first nor the last one
    index2=index #should be higher
    while index2<=index:
        index2 = random.randint(0, len(solution) - 1)
    # now we have the 2 indeces to use.
    selected_edge1 = solution[index]
    selected_edge2 = solution[index2]
    solution[index] = selected_edge2
    solution[index2] = selected_edge1
    #print("soluzione finale:", fitness(solution))
    return solution

In [63]:
def inversion_mutation(solution):
    # choose the index:
    index, index2 = np.random.randint(0, len(solution) - 1, size=2) #not the first nor the last one
    if index<index2:
        for _ in range(index, index2):
            sol=solution[index]
            solution[index]=solution[index+1]
            solution[index+1]=sol
    else:
        for _ in range(index2, index):
            sol=solution[index2]
            solution[index2]=solution[index2+1]
            solution[index2+1]=sol
    return solution

In [61]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]

#select first solution:
solution = []
city = 0
solution.append(city)
visited = []
visited.append(int(city))
while len(visited)<len(cities):
    c0, c1 = find_closest(segments, city, visited)
    solution.append(c1)
    visited.append(c1)
    city=c1
solution.append(solution[0])
print("distanza finale prima di HC è ", fitness(solution)[1]*-1)


#simulated annealing
temperatura_iniziale = 100
tasso_riscaldamento = 1.1 
it=0

#one out of five approach
miglioramenti_recenti = deque(maxlen=5) 
miglioramenti_richiesti = 1  

#stopping criteria:
miglioramenti_recenti_stop = deque(maxlen=5000)
miglioramenti_recenti_stop.append(True)

# Initial solution: greedy one!
x_corrente = solution
costo_corrente = fitness(x_corrente)
best_cost = costo_corrente
best_sol = x_corrente

temperatura = temperatura_iniziale
while it<20000:
    it+=1
    # tweak the solution
    rn = random.random()
    while rn > 0.5:
        x_nuovo = swap_mutation(x_corrente.copy())
        costo_nuovo = fitness(x_nuovo)
        rn/=1.7
    
    # variation of fitness by changing sign
    delta_costo = costo_nuovo[1]*(-1) - costo_corrente[1]*(-1) 
    #we are sure the solution after swap mutation is valid if the previous was it.
        
    if delta_costo < 0 or (random.random() < math.exp(-delta_costo / temperatura) and delta_costo!=0):
        x_corrente = x_nuovo
        costo_corrente = costo_nuovo
        miglioramenti_recenti.append(True)
        miglioramenti_recenti_stop.append(True)
        if costo_corrente[1]*(-1) < best_cost[1]*(-1):
            best_cost = costo_corrente
            best_sol = x_corrente
    else:
        miglioramenti_recenti.append(False)
        miglioramenti_recenti_stop.append(False)

    if miglioramenti_recenti.count(True) > miglioramenti_richiesti:
        temperatura *= tasso_riscaldamento  # more exploration
    if miglioramenti_recenti.count(True) < miglioramenti_richiesti:
        temperatura /= tasso_riscaldamento
    
    if miglioramenti_recenti_stop.count(True)==0: #stop condition
        break

fitness_new=fitness(best_sol)[1]*(-1)
print("solution after SA is: ", fitness_new)


distanza finale prima di HC è  4436.03176952516
solution after SA is:  4431.363767937219


With inversion mutation:

In [71]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]

#select first solution:
solution = []
city = 4
solution.append(city)
visited = []
visited.append(int(city))
while len(visited)<len(cities):
    c0, c1 = find_closest(segments, city, visited)
    solution.append(c1)
    visited.append(c1)
    city=c1
solution.append(solution[0])
print("distanza finale prima di HC è ", fitness(solution)[1]*-1)


#simulated annealing
temperatura_iniziale = 100
tasso_riscaldamento = 1.1 
it=0

#one out of five approach
miglioramenti_recenti = deque(maxlen=5) 
miglioramenti_richiesti = 1  

#stopping criteria:
miglioramenti_recenti_stop = deque(maxlen=5000)
miglioramenti_recenti_stop.append(True)

# Initial solution: greedy one!
x_corrente = solution
costo_corrente = fitness(x_corrente)
best_cost = costo_corrente
best_sol = x_corrente

temperatura = temperatura_iniziale
while it<20000:
    it+=1
    # tweak the solution
    rn = random.random()
    while rn > 0.5:
        x_nuovo = inversion_mutation(x_corrente.copy())
        costo_nuovo = fitness(x_nuovo)
        rn/=1.7
    
    # variation of fitness by changing sign
    delta_costo = costo_nuovo[1]*(-1) - costo_corrente[1]*(-1) 
    #we are sure the solution after swap mutation is valid if the previous was it.
        
    if delta_costo < 0 or (random.random() < math.exp(-delta_costo / temperatura) and delta_costo!=0):
        x_corrente = x_nuovo
        costo_corrente = costo_nuovo
        miglioramenti_recenti.append(True)
        miglioramenti_recenti_stop.append(True)
        if costo_corrente[1]*(-1) < best_cost[1]*(-1):
            best_cost = costo_corrente
            best_sol = x_corrente
    else:
        miglioramenti_recenti.append(False)
        miglioramenti_recenti_stop.append(False)

    if miglioramenti_recenti.count(True) > miglioramenti_richiesti:
        temperatura *= tasso_riscaldamento  # more exploration
    if miglioramenti_recenti.count(True) < miglioramenti_richiesti:
        temperatura /= tasso_riscaldamento
    
    if miglioramenti_recenti_stop.count(True)==0: #stop condition
        break

fitness_new=fitness(best_sol)[1]*(-1)
print("solution after SA is: ", fitness_new)


distanza finale prima di HC è  4857.080036214828
solution after SA is:  4825.432135133487
